In [366]:
import pandas as pd
import numpy as np
import glob

In [367]:
df=pd.read_csv('femodel.csv', low_memory=False, )

In [368]:
df.drop_duplicates(inplace=True)

In [369]:
df['corruption_dummy'] = 0

df.loc[df.state == 'VA', ['corruption_dummy']] = 1
df.loc[df.state == 'SD', ['corruption_dummy']] = 1
df.loc[df.state == 'LA', ['corruption_dummy']] = 1
df.loc[df.state == 'IL', ['corruption_dummy']] = 1
df.loc[df.state == 'KY', ['corruption_dummy']] = 1
df.loc[df.state == 'AZ', ['corruption_dummy']] = 1
df.loc[df.state == 'OH', ['corruption_dummy']] = 1
df.loc[df.state == 'MD', ['corruption_dummy']] = 1
df.loc[df.state == 'AL', ['corruption_dummy']] = 1
df.loc[df.state == 'FL', ['corruption_dummy']] = 1

df['corruption_amtsum']=df.corruption_dummy * df.amtsum
df['corruption_amtsumlag1']=df.corruption_dummy * df.amtsumlag_1
df['corruption_amtsumfwd1']=df.corruption_dummy * df.amtsumfwd_1
df['corruption_amtsumdff']=df.corruption_dummy * df.amtsumdff
df['corruption_amtsumlag_dff']=df.corruption_dummy * df.amtsumlag_dff
df['corruption_amtsumfwd_dff']=df.corruption_dummy * df.amtsumfwd_dff

In [370]:
df.columns.tolist()

['date',
 'years_exp',
 'cid',
 'cfcontrol',
 'leg_totalbills',
 'state',
 'amtsumlag_1',
 'amtsumfwd_1',
 'unemployment',
 'billsum',
 'amtsum',
 'Republican',
 'sen',
 'sen_amtsum',
 'sen_amtsumlag1',
 'sen_amtsumfwd1',
 'Republican_amtsum',
 'Republican_amtsumlag1',
 'Republican_amtsumfwd1',
 'unemplag_1',
 'post2009_years',
 'logleg_totalbills',
 'logamtsumlag_1',
 'logamtsumfwd_1',
 'logamtsum',
 'amtsum_post2009years',
 'amtsumfwd1_post2009years',
 'amtsumlag1_post2009years',
 'logamtsum_post2009years',
 'logamtsumfwd1_post2009years',
 'logamtsumlag1_post2009years',
 'amtsumdff',
 'logamtsumdff',
 'billsumdff',
 'logbillsumdff',
 'amtsumfwd_dff',
 'amtsumlag_dff',
 'logamtsumlag_dff',
 'leg_totalbillsdff',
 'years_expdff',
 'amtsumfwd1_post2009yearsdff',
 'amtsumlag1_post2009yearsdff',
 'amtsum_post2009yearsdff',
 'republican_amtsumlagdff',
 'republican_amtsumfwd1dff',
 'sen_amtsumlag1dff',
 'sen_amtsumfwd1dff',
 'sen_amtsumdff',
 'cfcontroldff',
 'unemploymentdff',
 'corruption_

In [373]:
df.isnull().sum()

date                           0
years_exp                      0
cid                            0
cfcontrol                      0
leg_totalbills                 0
state                          0
amtsumlag_1                    0
amtsumfwd_1                    0
unemployment                   0
billsum                        0
amtsum                         0
Republican                     0
sen                            0
sen_amtsum                     0
sen_amtsumlag1                 0
sen_amtsumfwd1                 0
Republican_amtsum              0
Republican_amtsumlag1          0
Republican_amtsumfwd1          0
unemplag_1                     0
post2009_years                 0
logleg_totalbills              0
logamtsumlag_1                 0
logamtsumfwd_1                 0
logamtsum                      0
amtsum_post2009years           0
amtsumfwd1_post2009years       0
amtsumlag1_post2009years       0
logamtsum_post2009years        0
logamtsumfwd1_post2009years    0
logamtsuml

In [375]:
df.to_csv('remodel.csv', index=False)

In [343]:


dy=df[['years_exp', 'cfcontrol', 'leg_totalbills', 'amtsum', 'billsum', 'unemployment', 'date', 'cid']]

dy.to_csv('descriptive statistics.csv')
#df = df.drop_duplicates('bills', keep='first')

In [344]:
dy.drop_duplicates(inplace=True) 
#

/home/jayaramdas/anaconda3/lib/python3.5/site-packages/pandas/util/decorators.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return func(*args, **kwargs)


## Deriving corruption dummy.

In [345]:
sa=pd.read_csv('Thesis/ExternalData/state_abbreviation.csv')

fedconvictions=pd.read_csv('Thesis/ExternalData/No.FederalEmployees/convictions.csv')

In [346]:
fedconvictions[['date']]= fedconvictions[['date']].astype('int64')
#test=pd.read_csv('Thesis/ExternalData/No. Federal Employees/14stall.csv')
sa.rename(columns={'state' : 'abbv'}, inplace=True)
sa.rename(columns={'GeoName' : 'state'}, inplace=True)

In [347]:
fedemployees = pd.concat((pd.read_csv(f, header=0, names=['state','function','employees', 'date'])\
for f in glob.glob('Thesis/ExternalData/No.FederalEmployees/**stall.csv')))

In [348]:
fedemp=fedemployees[(fedemployees['function'] == 'Total') & (fedemployees['state'] != 'United States')]
#fedemp=fedemp[(fedemp['GeoName'] != 'US')]


In [349]:
#fedemp.rename(columns={'GeoName':'state'}, inplace=True)
fedmerge=pd.merge(fedconvictions, fedemp, on=('state', 'date'), how='right')
#fedemp.drop('GeoName', axis=1, inplace=True)

fedemp.dtypes

state        object
function     object
employees     int64
date          int64
dtype: object

In [350]:
fedmerge['conviction_per_pubemployee']=fedmerge.convictions / fedmerge.employees

fedmerge['sum_conviction_pubemployee'] =fedmerge.groupby('state').conviction_per_pubemployee.transform('sum')

In [351]:
fedmerge.drop(['function', 'employees', 'convictions', 'conviction_per_pubemployee', 'date'],\
                     axis=1, inplace=True)

In [352]:
fedmerge.drop_duplicates(keep='first', inplace=True)


In [353]:
fedmergewithstate=pd.merge(sa, fedmerge, on='state', how='inner')
sa.rename(columns={'state' : 'abbv'}, inplace=True)
#fedmerge.sort_values('sum_conviction_pubemployee', ascending=False).head(10)

In [354]:
fedmergewithstate.drop(['state'],\
                     axis=1, inplace=True)
fedmergewithstate.rename(columns={'abbv' : 'state'}, inplace=True)

## These are the states with the top 10 number of corruption convictions per  federal employee.

In [355]:
fedmergewithstate.sort_values('sum_conviction_pubemployee', axis=0, ascending=False).head(10)

,state,sum_conviction_pubemployee
40,SD,0.004591
17,LA,0.003973
45,VA,0.003448
12,IL,0.003329
16,KY,0.003097
2,AZ,0.002871
34,OH,0.002859
19,MD,0.002779
0,AL,0.002755
8,FL,0.002463
